### TITANIC MACHINE LEARNING CHALLENGE

Import the necessary modules and files

In [10]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from random import random

file_path_train = 'D:/WEB/train.csv'
file_path_test = 'D:/WEB/test.csv'

data = pd.read_csv(file_path_train)
test = pd.read_csv(file_path_test)

Preview the first 5 rows and columns headers

In [3]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Get list of columns with missing values

In [7]:
[col for col in data.columns if data[col].isnull().sum() > 0]

['Age', 'Cabin', 'Embarked']

At first we will exclude PassengerId, Name, Ticket and Cabin

In [12]:
features = ["Pclass","Sex","Age","SibSp","Parch","Fare","Embarked"]

Not a standard method but just a simple way of fillng the missing values in the Age column

In [13]:


X = data[features]
y = data["Survived"]
l = test["Age"].min()
u = test["Age"].max()
test["Age"] = test["Age"].fillna(l+(u-l)*random())
x_test = test[features]

X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.3,random_state=0,stratify=y)


We will impute the missing values in the embarked column using the most frequent strategy

In [14]:
smp = SimpleImputer(strategy='most_frequent')
imputed_X_train = pd.DataFrame(smp.fit_transform(X_train))
imputed_X_val = pd.DataFrame(smp.transform(X_val))
imputed_X_train.columns = X_train.columns
imputed_X_val.columns = X_val.columns

Using one hot encoding we will transform our categorical features to numeric

In [15]:
#get the columns with categorical values
cat = X.select_dtypes(include="object")
onh = OneHotEncoder(handle_unknown='ignore',sparse=False)
cat_X_train = pd.DataFrame(onh.fit_transform(imputed_X_train[cat.columns]))
cat_X_val = pd.DataFrame(onh.transform(imputed_X_val[cat.columns]))
cat_X_test = pd.DataFrame(onh.transform(x_test[cat.columns]))
cat_X_train.index = imputed_X_train.index
cat_X_val.index = imputed_X_val.index
cat_X_test.index = x_test.index
num_X_train = imputed_X_train.drop(cat.columns,axis=1)
num_X_val = imputed_X_val.drop(cat.columns,axis=1)
num_X_test = x_test.drop(cat.columns,axis=1)
fX_train = pd.concat([num_X_train,cat_X_train],axis=1)
fX_val = pd.concat([num_X_val,cat_X_val],axis=1)
fX_test = pd.concat([num_X_test,cat_X_test],axis=1)

In [16]:
fX_train.head()

,Pclass,Age,SibSp,Parch,Fare,0,1,2,3,4
0,3,29,0,0,7.775,0.0,1.0,0.0,0.0,1.0
1,3,21,0,0,8.6625,0.0,1.0,0.0,0.0,1.0
2,3,22,1,0,16.1,0.0,1.0,0.0,0.0,1.0
3,2,17,0,0,12,1.0,0.0,1.0,0.0,0.0
4,3,49,0,0,0,0.0,1.0,0.0,0.0,1.0


The Random Forest classifier will be used

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier(n_estimators=100,random_state=0)
#we drop the fare column for now as one row has a missing value in the test data
a=fX_train.drop(["Fare"],axis=1)
b=fX_val.drop(["Fare"],axis=1)
c=fX_test.drop(["Fare"],axis=1)
rf.fit(a,y_train)

t = rf.predict(b)

print(accuracy_score(t,y_val))
#print(c["Age"].isnull())
predictions = rf.predict(c)

output = pd.DataFrame({'PassengerId': test.PassengerId, 'Survived': predictions})
output.to_csv('D:/WEB/my_submission.csv', index=False)



0.835820895522388


An accuracy of 83.58% is obtained using the validation data. The model can further be improved to increase this. Other models can also be used.